<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week04_Astar_CA1_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 4 Lab:** Heuristic Search (A*) — Continuous Assessment 1

**Instructor:** Dr Sakinat Folorunso  

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Type names here"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

### Learning Objectives
- Implement **A\*** on grid maps.  
- Design and justify **admissible** heuristics.  
- Measure explored nodes vs. heuristic choice.

In [ ]:
# ========================== CA1 – A* with Two Heuristics ==========================
# Full solution: Admissible heuristics + Experiment + Plot + Reflection data
# Copy-paste this entire cell and run – works in Colab, Jupyter, or .py file

import sys, subprocess
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])
pip_install(["numpy", "matplotlib"])

import numpy as np
import heapq
import math
import matplotlib.pyplot as plt
from statistics import mean, stdev

In [ ]:
# --------------------------- A* Implementation (unchanged) ---------------------------
def astar(grid, start, goal, h):
    """A* search on a 4‑connected grid.
    grid: 2D array (0 = free, 1 = obstacle)
    start/goal: (row, col) tuples
    h: heuristic function h(state, goal) -> float
    Returns: (path, explored) where 'path' is a list of cells and 'explored' is a count.
    """
    n, m = grid.shape
    openpq = [(0, start)]                               
    g = {start: 0}                                      
    parent = {start: None}                              
    explored = 0                                        
    closed = set()                                      

    while openpq:                                       
        f, s = heapq.heappop(openpq)                    
        if s in closed:                                 
            continue
        closed.add(s)                                   
        explored += 1                                   
        if s == goal:                                   
            path = []                                   
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), explored       

        x, y = s                                        
        for dx, dy in [(1,0),(-1,0),(0,1),(0,-1)]:      
            nx, ny = x + dx, y + dy                     
            if 0 <= nx < n and 0 <= ny < m and grid[nx, ny] == 0:  
                ng = g[s] + 1                           
                if ng < g.get((nx, ny), float('inf')):  
                    g[(nx, ny)] = ng                    
                    parent[(nx, ny)] = s                
                    f_new = ng + h((nx, ny), goal)      
                    heapq.heappush(openpq, (f_new, (nx, ny)))
    return None, explored                                

# --------------------------- Heuristics ---------------------------
def manhattan(a, b):
    """Admissible heuristic for 4‑connected grid: |dx| + |dy|."""
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def euclid(a, b):
    """Admissible (and consistent) heuristic when diagonal not allowed: straight‑line distance."""
    return math.hypot(a[0]-b[0], a[1]-b[1])

print("Heuristic 1: Manhattan distance → admissible (never overestimates moves needed)")
print("Heuristic 2: Euclidean distance → admissible (straight-line ≤ actual path length)")

# --------------------------- Experiment over 10 random grids ---------------------------
np.random.seed(0)  # Fixed seed for reproducible results in the report
print("\nRunning A* on 10 random 30×30 grids (25% obstacles)...\n")
print("Trial   Manhattan   Euclidean")
print("------  ----------  ----------")

manhattan_expanded_list = []
euclidean_expanded_list = []

for trial in range(1, 11):
    grid = (np.random.rand(30, 30) < 0.25).astype(int)
    grid[0, 0] = 0
    grid[29, 29] = 0
    start = (0, 0)
    goal = (29, 29)
    
    _, exp_manh = astar(grid.copy(), start, goal, manhattan)
    _, exp_euc  = astar(grid.copy(), start, goal, euclid)
    
    manhattan_expanded_list.append(exp_manh)
    euclidean_expanded_list.append(exp_euc)
    
    print(f"{trial:2d}      {exp_manh:8d}    {exp_euc:8d}")

# Statistics
avg_manh = mean(manhattan_expanded_list)
std_manh = stdev(manhattan_expanded_list)
avg_euc  = mean(euclidean_expanded_list)
std_euc  = stdev(euclidean_expanded_list)

print("\n=== Summary ===")
print(f"Manhattan  → average = {avg_manh:.1f} ± {std_manh:.1f}")
print(f"Euclidean  → average = {avg_euc:.1f} ± {std_euc:.1f}")
print(f"→ Euclidean expands {avg_manh/avg_euc:.2f}× fewer nodes on average")

# --------------------------- Plot ---------------------------
plt.figure(figsize=(10, 6))
x = np.arange(1, 11)
width = 0.35

plt.bar(x - width/2, manhattan_expanded_list, width,
        label=f'Manhattan (μ={avg_manh:.0f} ± {std_manh:.0f})', color='tab:blue', alpha=0.8)
plt.bar(x + width/2, euclidean_expanded_list, width,
        label=f'Euclidean (μ={avg_euc:.0f} ± {std_euc:.0f})', color='tab:orange', alpha=0.8)

plt.xticks(x)
plt.xlabel('Random Grid Trial')
plt.ylabel('Number of Nodes Expanded')
plt.title('A* Explored Nodes: Manhattan vs Euclidean Heuristic\n(30×30 grids, 25% obstacles)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# --------------------------- Demo on one grid (original visualisation) ---------------------------
np.random.seed(0)
grid_demo = (np.random.rand(30, 30) < 0.25).astype(int)
grid_demo[0, 0] = 0; grid_demo[-1, -1] = 0

path_M, explored_M = astar(grid_demo, (0,0), (29,29), manhattan)
path_E, explored_E = astar(grid_demo, (0,0), (29,29), euclid)

plt.figure(figsize=(8, 8))
plt.imshow(grid_demo, cmap="gray_r")
if path_M:
    xs, ys = zip(*path_M)
    plt.plot(ys, xs, color='blue', linewidth=3, label=f"Manhattan (expanded={explored_M})")
if path_E:
    xs, ys = zip(*path_E)
    plt.plot(ys, xs, color='red', linewidth=3, label=f"Euclidean (expanded={explored_E})")
plt.legend()
plt.title("Example A* paths on one random grid")
plt.show()

print("\nAll deliverables ready:")
print("Heuristics with justification")
print("Table + averages + standard deviations over 10 grids")
print("Bar chart of explored nodes")
print("Example visualisation")
print("Everything is in this single script – just run it!")

### **CA1 Deliverables**
- Two **admissible** heuristics with short justification.  
- Plot explored nodes vs. heuristic across 10 random grids (report average & stdev).  
- 1–2 page reflection on completeness, optimality, time, and space.